In [1]:
import sys
sys.path.append('../')


data_path = "/home/ubuntu/repos/Efficient-MT/data/wmt_2020_all_reduced_system_class_updated_predicted_categories_best_mt.csv"

In [2]:
import pandas as pd
df = pd.read_csv(data_path)
df.head(11)

,seg_id,system,sample_id,source,target,category,severity,source_topic,target_topic,Num token,...,"COMET_QE, uncased, punctuated","COMET_QE, uncased, not punctuated",CLSSS,topic_distance,labse_distance,predicted_categories,translation_quality,postedit,post_edit_translation_quality,best_mt
0,3,MT A,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, jetzt ein Bauträger und Inhaber e...","['Other', 'Accuracy', 'Terminology', 'Fluency']",Edit,Sports,Sports,0.0,...,0.07130,0.0,86.240,0.028171,0.119782,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, jetzt ein Immobilienentwickler u...",95.0,MT B
1,3,MT B,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, der heute als Bauunternehmer und ...","['Accuracy', 'Fluency']",Edit,Sports,Entertainment & Music,10.0,...,0.08610,0.0,85.480,0.094228,0.122524,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, der heute als Immobilienentwickl...",95.0,MT B
2,3,MT C,3_3,"Matt Fiddes, now a property developer and owne...","Jackson sei der Überzeugung gewesen, dass sein...","['Accuracy', 'Terminology']",Edit,Sports,Sports,7.0,...,0.00010,0.0,78.420,0.074360,0.186550,"['fluency', 'accuracy']",85.0,"""Matt Fiddes, der inzwischen Bauträger und Eig...",95.0,MT B
3,3,MT D,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, jetzt ein Immobilienentwickler un...","['Other', 'Accuracy', 'Terminology', 'Fluency']",Edit,Sports,Entertainment & Music,-1.0,...,0.00040,0.0,88.570,0.143223,0.101885,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, jetzt ein Immobilienentwickler u...",95.0,MT B
4,3,MT E,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, jetzt Immobilienentwickler und Be...","['Accuracy', 'Terminology', 'Fluency']",Edit,Sports,Entertainment & Music,-2.0,...,0.00000,0.0,88.990,0.081306,0.095368,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, jetzt Immobilienentwickler und B...",98.0,MT B
5,3,MT F,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, heute Immobilienentwickler und Be...","['Accuracy', 'Terminology', 'Fluency']",Edit,Sports,Entertainment & Music,-6.0,...,0.00000,0.0,0.000,0.125584,0.096042,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, heute Immobilienentwickler und B...",95.0,MT B
6,3,MT G,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, jetzt ein Immobilienentwickler un...","['Accuracy', 'Terminology', 'Fluency']",Edit,Sports,Sports,-5.0,...,0.00000,0.0,0.000,0.007942,0.112301,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, jetzt ein Immobilienentwickler u...",98.0,MT B
7,3,MT H,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, jetzt ein Immobilienentwickler un...","['Accuracy', 'Terminology', 'Fluency']",Edit,Sports,Entertainment & Music,-1.0,...,0.00000,0.0,88.820,0.107610,0.098497,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, jetzt ein Immobilienentwickler u...",95.0,MT B
8,3,MT I,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, jetzt Immobilienentwickler und Be...","['Other', 'Accuracy', 'Terminology', 'Fluency']",Edit,Sports,Entertainment & Music,-4.0,...,0.00000,0.0,0.000,0.081504,0.100169,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, jetzt Immobilienentwickler und B...",95.0,MT B
9,3,MT J,3_3,"Matt Fiddes, now a property developer and owne...","Matt Fiddes, heute Immobilienentwickler und Be...","['No-error', 'Accuracy', 'Terminology', 'Fluen...",Edit,Sports,Sports,-3.0,...,0.00020,0.0,89.050,0.033303,0.093062,"['fluency', 'accuracy']",95.0,"""Matt Fiddes, heute Immobilienentwickler und B...",95.0,MT B


## Post Edit Analysis

In [3]:
# get difference after post edit
def get_diff_after_post_edit(row):
    return row["post_edit_translation_quality"] - row["translation_quality"]

diff = df.apply(get_diff_after_post_edit, axis=1)


In [4]:
diff.mean(), diff.std()

(4.3341003221352965, 10.253209778141947)

In [5]:
# get post abs diff in number of caracters in post edit and target
def get_diff_after_post_edit(row):
    return abs(len(row["postedit"]) - len(row["target"]))
diff = df.apply(get_diff_after_post_edit, axis=1)
diff.mean(), diff.std()

(16.347546012269937, 143.75079127588495)

## Comet vs Translation Quality

In [8]:
# get correlation between translation quality and commet COMET_QE, uncased, not punctuated

spearman = df["translation_quality"].corr(df["COMET_QE, cased, punctuated"], method='spearman')
pearson = df["translation_quality"].corr(
    df["COMET_QE, cased, punctuated"], method="pearson"
)
kendall = df["translation_quality"].corr(
    df["COMET_QE, cased, punctuated"], method="kendall"
)

print(
    f"{"COMET_QE, cased, punctuated"} Spearman: {spearman}, Pearson: {pearson}, Kendall: {kendall}"
)

COMET_QE, cased, punctuated Spearman: 0.40366448909010194, Pearson: 0.3264613179125817, Kendall: 0.3157994723980547


## Accuracy in best model selection 

In [11]:
# best model can be selected by highest comet
# for each segment get the best model

best_models_true = []
best_models_pred = []
for seg_id in df["seg_id"].unique():
    seg = df[df["seg_id"] == seg_id]
    best_model_idx = seg["COMET_QE, cased, punctuated"].idxmax()
    best_model_name = seg.loc[best_model_idx]["system"]
    best_models_true.append(best_model_name)
    best_models_pred.append(seg["best_mt"].iloc[0])

In [25]:
# how to find top k accuracy
# for each segment get the best model

import numpy as np
def get_accuracy(k):
    accuracy_list = []
    for seg_id in df["seg_id"].unique():
        seg = df[df["seg_id"] == seg_id]
        best_model_idx = seg["COMET_QE, cased, punctuated"].nlargest(k).index
        best_model_name = seg.loc[best_model_idx]["system"]
        if seg["best_mt"].iloc[0] in best_model_name.to_list():
            accuracy_list.append(1)
        else:
            accuracy_list.append(0)

    return np.mean(accuracy_list)

In [28]:
get_accuracy(1), get_accuracy(3)

(0.23773006134969324, 0.5720858895705522)

## Evaluate Category List
Here we have category and predicted_category columns. We will evaluate the accuracy of the model by comparing these two columns in which we have multiple categories.

In [37]:
category = df["category"].to_list()
# need to convert string to list
category = [eval(c) for c in category]
# make lower case
category = [[c.lower() for c in cat] for cat in category]



In [42]:
predicted_category = df["predicted_categories"].to_list()
# need to convert string to list
predicted_category = [eval(c) for c in predicted_category]
# "unknown" should be mapped to other"
predicted_category = [[c.lower() if c != "unknown" else "other" for c in cat] for cat in predicted_category]

In [43]:
#make a milticlass classification report
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(category)
y_true = mlb.transform(category)
y_pred = mlb.transform(predicted_category)
print(classification_report(y_true, y_pred, target_names=mlb.classes_))

                   precision    recall  f1-score   support

         accuracy       0.68      0.87      0.77      4341
          fluency       0.53      0.91      0.67      3265
locale convention       0.00      0.00      0.00        70
         no-error       0.00      0.00      0.00      3533
            other       0.45      0.02      0.04      2922
      terminology       0.25      0.02      0.03      1801

        micro avg       0.60      0.43      0.50     15932
        macro avg       0.32      0.30      0.25     15932
     weighted avg       0.41      0.43      0.36     15932
      samples avg       0.59      0.41      0.46     15932



/home/ubuntu/miniconda3/envs/evolvemt/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ubuntu/miniconda3/envs/evolvemt/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
